In [3]:
import math
import pandas as pd
from enum import Enum

class MessageKind(Enum):
    MEASURE = "MEASURE"
    GENREQ = "GENREQ"
    GENRESP = "GENRESP"
    
class ReqResKind(Enum):
    READ = "READ"
    COMMAND = "COMMAND"
    UNSET = "UNSET"
    
class AtkStatus(Enum):
    UNMOD = "UNMOD"
    COMPR = "COMPR"
    BLOCK = "BLOCK"
    
class EvilState(Enum):
    INACTIVE = "INACTIVE"
    MIDDLE = "MIDDLE"
    FULL = "FULL"
    
class Keys(Enum):
    ID = "id"
    MESSAGEKIND = "messageKind"
    REQRESKIND = "reqResKind"
    ATKSTATUS = "atkStatus"
    DATA = "data"
    EVILSTATE = "evilState"
    CREATIONTIME = "creationTime"
    TIMESTAMP = "timestamp"

class OutKeys(Enum):
    MITM = "MITM"
    SRM = "SRM"
    UC = "UC"
    UPS = "UPS"
    IMD = "IMD"
    MC = "MC"
    CC = "CC"
    
    
evilStateToVar = {
    EvilState.INACTIVE.value: 1,
    EvilState.MIDDLE.value: 2,
    EvilState.FULL.value: 3
}
    
# Simulation time limit in seconds
simTimeLimit = 120
# Number of runs
numRuns = 30
# Connection index (0 for server[0] etc...)
connIndex = 1
# Discretization time in seconds
deltaT = 5
# Number of levels for SRM and UC
numLevels = 4


relPath = "../logs/"
evilFileName = "evilClient.app[" + connIndex.__str__() + "].csv"
cliFileName = "client[0].app[" + connIndex.__str__() + "].csv"
serFileName = "server[" + connIndex.__str__() + "].app[0].csv"


In [4]:
for i in range(numRuns):
    complRelPath = relPath + "run" + i.__str__() + "/"
    evilDf = pd.read_csv(complRelPath + evilFileName)
    cliDf = pd.read_csv(complRelPath + cliFileName)
    evilDf = evilDf.drop(Keys.CREATIONTIME.value, axis=1)
    
    currT = 0
    MITMv = 0
    SRMv = 0
    UCv = 0
    MCv = 0
    CCv = 0
    outDf = pd.DataFrame(columns=[OutKeys.MITM.value, OutKeys.SRM.value, OutKeys.UC.value, OutKeys.MC.value, OutKeys.CC.value])
    while currT < simTimeLimit:
        evilWindowDf = evilDf[evilDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        cliWindowDf = cliDf[cliDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        evilTotNumWindow = evilWindowDf.shape[0]
        cliTotNumWindow = cliWindowDf.shape[0]
        if not evilWindowDf.empty:
            MITMv = evilStateToVar[evilWindowDf.iloc[-1][Keys.EVILSTATE.value]]
            numComprMeas = evilWindowDf[((evilWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value) | 
                                            (evilWindowDf[Keys.REQRESKIND.value] == ReqResKind.READ.value)) & 
                                            (evilWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            numComprCmds = evilWindowDf[(evilWindowDf[Keys.REQRESKIND.value] == ReqResKind.COMMAND.value) & 
                                            (evilWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            SRMv = max(math.trunc((numComprMeas/evilTotNumWindow) * 4), SRMv)
            UCv = max(math.trunc((numComprCmds/evilTotNumWindow) * 4), UCv)
        if not cliWindowDf.empty:
            numComprMeas = cliWindowDf[((cliWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value) | 
                                            (cliWindowDf[Keys.REQRESKIND.value] == ReqResKind.READ.value)) & 
                                            (cliWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            numComprCmds = cliWindowDf[(cliWindowDf[Keys.REQRESKIND.value] == ReqResKind.COMMAND.value) & 
                                            (cliWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            MCv = max(math.trunc((numComprMeas/cliTotNumWindow) * 4), MCv)
            CCv = max(math.trunc((numComprCmds/cliTotNumWindow) * 4), CCv)
        outDf.loc[len(outDf.index)] = [MITMv, SRMv, UCv, MCv, CCv]

        currT += deltaT
    print(outDf)
    outDf.to_csv("outTrain.csv", index=False)


    MITM  SRM  UC  MC  CC
0      0    0   0   0   0
1      1    0   0   0   0
2      1    0   0   0   0
3      1    0   0   0   0
4      1    0   0   0   0
5      1    0   0   0   0
6      1    0   0   0   0
7      1    0   0   0   0
8      1    0   0   0   0
9      1    0   0   0   0
10     1    0   0   0   0
11     1    0   0   0   0
12     1    0   0   0   0
13     2    0   0   0   0
14     2    0   0   0   0
15     2    0   0   0   0
16     2    0   0   0   0
17     2    0   0   0   0
18     2    0   0   0   0
19     2    0   0   0   0
20     2    0   0   0   0
21     2    0   0   0   0
22     3    1   0   0   0
23     3    1   0   0   0
    MITM  SRM  UC  MC  CC
0      0    0   0   0   0
1      1    0   0   0   0
2      1    0   0   0   0
3      1    0   0   0   0
4      1    0   0   0   0
5      1    0   0   0   0
6      1    0   0   0   0
7      1    0   0   0   0
8      1    0   0   0   0
9      1    0   0   0   0
10     1    0   0   0   0
11     2    0   0   0   0
12     2    

In [5]:
# Initialize columns labels
columns = []
for key in OutKeys:
    columns.append(key.value)
    for j in range(1, (simTimeLimit // deltaT)+1):
        columns.append(key.value + "_" + j.__str__())
outDf = pd.DataFrame(columns=columns)  

for i in range(numRuns):
    complRelPath = relPath + "run" + i.__str__() + "/"
    evilDf = pd.read_csv(complRelPath + evilFileName)
    cliDf = pd.read_csv(complRelPath + cliFileName)
    evilDf = evilDf.drop(Keys.CREATIONTIME.value, axis=1)
    cliDf = cliDf.drop(Keys.CREATIONTIME.value, axis=1)
    
    currT = 0
    MITMv = 0
    SRMv = 0
    UCv = 0
    MCv = 0
    CCv = 0
    while currT < simTimeLimit:
        evilWindowDf = evilDf[evilDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        cliWindowDf = cliDf[cliDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        evilTotNumWindow = evilWindowDf.shape[0]
        cliTotNumWindow = cliWindowDf.shape[0]
        if not evilWindowDf.empty:
            MITMv = evilStateToVar[evilWindowDf.iloc[-1][Keys.EVILSTATE.value]]
            numComprMeas = evilWindowDf[((evilWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value) | 
                                            (evilWindowDf[Keys.REQRESKIND.value] == ReqResKind.READ.value)) & 
                                            (evilWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            numComprCmds = evilWindowDf[(evilWindowDf[Keys.REQRESKIND.value] == ReqResKind.COMMAND.value) & 
                                            (evilWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            SRMv = max(math.trunc((numComprMeas/evilTotNumWindow) * numLevels), SRMv)
            if SRMv == numLevels: SRMv = numLevels - 1
            UCv = max(math.trunc((numComprCmds/evilTotNumWindow) * numLevels), UCv)
            if UCv == numLevels: UCv = numLevels - 1
        if not cliWindowDf.empty:
            numComprMeas = cliWindowDf[((cliWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value) | 
                                            (cliWindowDf[Keys.REQRESKIND.value] == ReqResKind.READ.value)) & 
                                            (cliWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            numComprCmds = cliWindowDf[(cliWindowDf[Keys.REQRESKIND.value] == ReqResKind.COMMAND.value) & 
                                            (cliWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            MCv = max(math.trunc((numComprMeas/cliTotNumWindow) * numLevels), MCv)
            if MCv == numLevels: MCv = numLevels - 1
            CCv = max(math.trunc((numComprCmds/cliTotNumWindow) * numLevels), CCv)
            if CCv == numLevels: CCv = numLevels - 1
        
        # Assign values
        numSlice = currT // deltaT    
        outDf.at[i, OutKeys.MITM.value] = OutKeys.MITM.value + "0"
        outDf.at[i, OutKeys.MITM.value + "_" + numSlice.__str__()] = OutKeys.MITM.value + MITMv.__str__()
        outDf.at[i, OutKeys.SRM.value] = OutKeys.SRM.value + "0"
        outDf.at[i, OutKeys.SRM.value + "_" + numSlice.__str__()] = OutKeys.SRM.value + SRMv.__str__()
        outDf.at[i, OutKeys.UC.value] = OutKeys.UC.value + "0"
        outDf.at[i, OutKeys.UC.value + "_" + numSlice.__str__()] = OutKeys.UC.value + UCv.__str__()
        outDf.at[i, OutKeys.MC.value] = OutKeys.MC.value + "0"
        outDf.at[i, OutKeys.MC.value + "_" + numSlice.__str__()] = OutKeys.MC.value + MCv.__str__()
        outDf.at[i, OutKeys.CC.value] = OutKeys.CC.value + "0"
        outDf.at[i, OutKeys.CC.value + "_" + numSlice.__str__()] = OutKeys.CC.value + CCv.__str__()

        currT += deltaT
    print(outDf)
    outDf.to_csv("outTrain.csv", index=False)


    MITM MITM_1 MITM_2 MITM_3 MITM_4 MITM_5 MITM_6 MITM_7 MITM_8 MITM_9  ...  \
0  MITM0  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  ...   

  CC_20 CC_21 CC_22 CC_23 CC_24 MITM_0 SRM_0 UC_0 MC_0 CC_0  
0   CC0   CC0   CC0   CC0   NaN  MITM0  SRM0  UC0  MC0  CC0  

[1 rows x 180 columns]
    MITM MITM_1 MITM_2 MITM_3 MITM_4 MITM_5 MITM_6 MITM_7 MITM_8 MITM_9  ...  \
0  MITM0  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  ...   
1  MITM0  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  ...   

  CC_20 CC_21 CC_22 CC_23 CC_24 MITM_0 SRM_0 UC_0 MC_0 CC_0  
0   CC0   CC0   CC0   CC0   NaN  MITM0  SRM0  UC0  MC0  CC0  
1   CC1   CC1   CC1   CC1   NaN  MITM0  SRM0  UC0  MC0  CC0  

[2 rows x 180 columns]
    MITM MITM_1 MITM_2 MITM_3 MITM_4 MITM_5 MITM_6 MITM_7 MITM_8 MITM_9  ...  \
0  MITM0  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  ...   
1  MITM0  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  MITM1  ...   
